--------------------------------------------------------------------------------------

* Team member names: Nathaniel del Rosario
* Team member IDs: A17562063

--------------------------------------------------------------------------------------

# Mini-project 4, DSC 170, Winter 2024

# Suitability Modeling

This project will focus on __suitability analysis__ with raster data. Your tasks will be both conceptual-level and technical. 

There are three parts to this assignment.

1. At the conceptual level, you will define a suitability model of your choice, for an area of your choice (preferably San Diego, because we already have worked with some local data). Consult the lectures on map combination, and also see https://pro.arcgis.com/en/pro-app/latest/help/analysis/spatial-analyst/suitability-modeler/the-general-suitability-modeling-workflow.htm for a brief description of what a suitability model is. For example, you may be looking for best areas for community gardens. Such areas are often selected from underutilized land in residential land uses, with good soils, good drainage, accessible (not steep slope), etc. So you would be looking for areas with a specific type of land use/land cover, with an appropriate range of values of slope, etc. You may build additional criteria based on a range of precipitaiton values, whether the area is affected by wildfires, or has low levels of soil erosion, etc. Feel free to use the imagery layers we explored or mentioned during raster-focused lectures. Several cells in lecture notebooks contained URLs to imagery layer collections available through AGOL - but feel free to find more. Also, feel free to download additional raster layers from elsewhere (an example in lecture demonstrates how to do this from the USGS image repository, and also how to get Sentinel-2 from ESA), publish them on ArcGIS Enterprise, and use in your model. 

You can use any __two__ of the <i>map combination</i> techniques discussed during lectures. You should identify the ones you use and discuss any uncertainty issues associated with these specific map combination models. 

As the outcome of this part, you will need to: a) describe the suitability model you want to develop; b) identify the raster data layers you will use; and c) describe two of the map combination techniques you will use to derive the two suitablity maps, and their pros and cons.

2. The second part will involve implementing your suitability model using arcgis raster functions. Many of these functions are new and experimental! Examples of what works are in the lecture notebooks. Be creative!    

3. The third part will be a brief write-up comparing the two output rasters generated for the suitability models using the two map combination techniques. 

The notebook should include documentation of the steps, as usually.



**Due Date: 03/04/2024 11:59PM (Pacific Time)**

**Total Possible Points: 30 Pts**

## Task 1: Formulate a suitability or risk model (5 points)

Before finding a dataset, we will form our question in the following way:
What are the best areas for _ / best areas that satisfy condition _ ?

### Question:

What are the best locations for new student housing in Berkeley?

### Data
- [Berkeley zoning data](https://data.cityofberkeley.info/browse?q=zoning&sortBy=relevance)
- [Population By Council District](https://geodata.lib.berkeley.edu/catalog/berkeley-s7wq14)
- [GeoData](https://geodata.lib.berkeley.edu/?utf8=%E2%9C%93&search_field=all_fields&q=berkeley)
- [Alameda County Open Data](https://data.acgov.org/search?collection=Dataset&source=AlamedaCounty.CA.US)
- [Fire Hazard Raster](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?layers=cdedfa4417f54ae0a274c78f7c9b8c8e)
- [Census Tracts](https://data.census.gov/allq=alameda%20&t=Housing:Housing%20Units:Renter%20Costs&g=050XX00US06001$1400000)
- [Berkeley Crime Data](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=9f67de4e3a0d49c39ae07034f5c32966)
- [Berkeley Crime Heatmap](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=c38a13f955c54877bd5c74e862237132)
- [Berkeley Bus Routes](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=761a98de8a1a435aad5f229692c5d229)
- [Berkeley Bike Lanes](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?layers=e1453a81e05f45dd9c44699c5b1a40b7)
- [Berkeley Bike Parking](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=9fa9fa7139584d359a0542cc4561d427)

### Important Features
- Population density
- Proximity to bike paths / bus routes
- Is it in a heat island?
- Proximity to frequent crime scenes 


## Task 2: Implement the model (20 points)

In [ ]:
# Imports, etc.

In [ ]:
# List imagery layers to be used in your model. 
# This cell should contian layer definitions.



In [ ]:
# Derive the area of interest (AOI) and its geometry and extent. 
# The smaller the area the better (so that you don't run into raster size limitations)


### Name the two map combination techniques you will use to combine the data and describe their pros and cons

Your text here 

In [ ]:
# Prepare your input layers for map combination: clip to AOI, remap/normalize, visualize the layers. 


In [ ]:
# Generate a composite raster layer for your first map combination technique


In [ ]:
# Generate a composite raster layer for your second map combination technique


## Task 3: Compare the results (5 points)
... and describe how different combination techniques resulted in different outputs (or not.) 

Your text here


In [ ]:
## Timekeeping
# Please let us know how much time you spent on this project, in hours: 
# (we will only examine distributions and won't look at individual responses)

assignment_timespent = 0